"""
Example of creating an RC model of a single zone fully deterministic
building from YABML in CDCM.

Author(s):
    Sreehari Manikkan

Date:
    05/16/2022
"""

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from yabml import *
from cdcm import *

from building_system import building_sys
from single_zone import single_zone_building

In [2]:
df = pd.read_csv("./rc_system_data/weather_data_2017_pandas.csv")

weather_sys = make_data_system(
    df[["Tout", "Qsg", "Qint"]],
    name="weather_sytem",
    column_units=["degC", "Wh", "Wh"],
    column_desciptions=[
        "Outdoor air temperature",
        "Solar irradiance",
        "Internal heat gain"
    ]
)

clock = make_clock(1800)

In [3]:
if os.path.exists('test1_rc.h5'):
    os.remove('test1_rc.h5')

for sample_building in single_zone_building.sample(1):
    sample_building_sys = building_sys(sample_building, weather_sys, clock)
    
    for sample_zone_sys in sample_building_sys:
        sample_zone_sys.forward()
        
        max_steps = 500
        saver = SimulationSaver("test1_rc.h5",
                                sample_zone_sys,
                                max_steps=max_steps
        )

        for i in range(max_steps):
            sample_zone_sys.forward()
            sample_zone_saver.save()
            sample_zone_sys.transition()

        #T_room_sensor will be a numpy array
        T_room_sensor = (
            saver.file_handler["/everything/zone_rc_sys/T_room_sensor"][:]
        )
        T_env = (
            saver.file_handler["/everything/zone_rc_sys/T_env"][:]
        )
        T_genv = (
            saver.file_handler["/everything/zone_rc_sys/T_genv"][:]
        )
        plt.plot(T_room_sensor, label='T_room_sensor')
        plt.plot(T_env, label='T_env')
        plt.plot(T_genv, label='T_genv')
        plt.legend()
        plt.show()

ValueError: Node energy has an uninitialized value. Please specify a value so that I can figure out the type of the variable I need to store.